In [1]:
from queue import PriorityQueue
import queue
import copy
directions = [(-1,0),(1,0),(0,-1),(0,1)]
class Node:
    def __init__(self,state,parent=None,anato_i_pos = 0,anato_j_pos = 0,path=[], has_weapon = False,cost = 0):
        self.state= state
        self.parent = parent
        self.path = path
        self.cost = cost
        self.anato_i_pos = anato_i_pos
        self.anato_j_pos = anato_j_pos
        self.has_weapon = has_weapon

    def __lt__(self, other):
        return self.cost < other.cost 
    

   

def read_map(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        map_data = []
        for line_num, line in enumerate(lines):
            row = []
            for char_num, char in enumerate(line.strip()):
                if char == '+':
                    row.append('1')
                elif char == '|':
                    row.append('1')

                else:
                    row.append(char)
            map_data.append(row)
        # Replace '0' in odd rows with columns at positions [4, 8, 12, ...]
        for i in range(len(map_data)):
            if i % 2 != 0:  # Check if row index is odd
                for j in range(4, len(map_data[i]), 4):  # Iterate through columns at positions [4, 8, 12, ...]
                    if map_data[i][j] not in ['A', 'S', 'z', 'w','1']:
                        map_data[i][j] = '0'  
        for i in range(len(map_data)):
            map_data[i] = ''.join(map_data[i])
    # Perform replacements with conditions
            if '---' in map_data[i]:
              map_data[i] = map_data[i].replace('---', '1')
            if ' A ' in map_data[i]:
             map_data[i] = map_data[i].replace(' A ', 'A')
            if 'A' in map_data[i]:
             map_data[i] = map_data[i].replace('A', 'A') 
            if 'z' in map_data[i]:
             map_data[i] = map_data[i].replace('z', 'z')
            if ' z ' in map_data[i]:
             map_data[i] = map_data[i].replace(' z ', 'z')
            if ' S ' in map_data[i]:
             map_data[i] = map_data[i].replace(' S ', 'S')
            if 'S' in map_data[i]:
             map_data[i] = map_data[i].replace('S', 'S') 
            if ' w ' in map_data[i]:
              map_data[i] = map_data[i].replace(' w ', 'w')
            if 'w' in map_data[i]:
              map_data[i] = map_data[i].replace('w', 'w')
            if '   ' in map_data[i]:
        
             map_data[i] = map_data[i].replace('   ', '0')
    # Convert back to list
            map_data[i] = list(map_data[i])
           
        

    return map_data

def valid_space(new_i,new_j,maze):
      
   return 0 <= new_i < len(maze) and 0 <= new_j < len(maze[0]) and maze[new_i][new_j] != '1'
             

    
def new_states(maze):
    
    newstates = []
    print("in new states function:")
    blnk_i,blnk_j = find_anto(maze)
    for dir in directions:
        new_position_i,new_position_j = blnk_i + dir[0], blnk_j+dir[1]
        if valid_space(new_position_i, new_position_j, maze):

            if maze[new_position_i][new_position_j] == '0':
                #swap places
                old_value = maze[new_position_i][new_position_j]
                temp = copy.deepcopy(maze)     
                temp[blnk_i][blnk_j] = old_value
                temp[new_position_i][new_position_j]= 'A'                                       
                newstates.append(temp)
            elif maze[new_position_i][new_position_j] == 'w':
                old_value = maze[new_position_i][new_position_j]
                temp = copy.deepcopy(maze)     
                temp[blnk_i][blnk_j] = 0
                temp[new_position_i][new_position_j]= 'A'                                       
                newstates.append(temp)
            else:
                old_value = maze[new_position_i][new_position_j]
                temp = copy.deepcopy(maze)     
                temp[blnk_i][blnk_j] = 0
                temp[new_position_i][new_position_j]= 'A'                                       
                newstates.append(temp)

          
    return newstates  

def remove_weapon(i,j,maze):
    i_coordinate = i
    j_coordinate = j

    maze[i_coordinate][j_coordinate] = 'A'

    return maze

def dfs(maze):
    root = Node(maze)
    a_i, a_j = find_anto(maze)
    visited = set()
    root.anato_i_pos, root.anato_j_pos = a_i, a_j
    stack = []
    #stack.push(root)
    stack.append(root)
    count = 0
    maze2 = copy.deepcopy(maze)
    while stack:
      
      current = stack.pop()
      current_i = current.anato_i_pos
      current_j = current.anato_j_pos
      count = count+1
      for row in current.state:
       print(''.join(str(cell) for cell in row))

      print()
      if maze2[current_i][current_j] == 'S':
        print(" found Safe House")
        return current.path
      if maze2[current_i][current_j] == 'w':
        current.has_weapon = True
        current.state = copy.deepcopy(remove_weapon(current_i,current_j, current.state))
        print(" found weapon")
        for row in current.state:
         print(''.join(str(cell) for cell in row))
      if maze2[current_i][current_j] == 'z':  
         if if_weapon(current) == True: #kill zombie here if has_weapon in node is true
               current.state = copy.deepcopy(kill_zombie(current_i,current_j,current.state))
         else: #kill_zombie is false to kill anato
             print("Anato killed!")
             return False 
      visited.add(tuple(map(tuple, current.state)))       
      newstateslist= new_states(current.state)   

      for st in newstateslist:
         if tuple(map(tuple, st)) not in visited:
             temp_node = Node(st)
             print("in loop:")
             ai,aj = find_anto(st)
             temp_node.anato_i_pos, temp_node.anato_j_pos = ai,aj
             temp_node.parent = current
             temp_node.has_weapon = current.has_weapon
             temp_node.path = current.path + [st]
             stack.append(temp_node)
    return None          


def bfs(maze):

    root = Node(maze)
    a_i, a_j = find_anto(maze)
    visited = set()
    root.anato_i_pos, root.anato_j_pos = a_i, a_j
    qu=queue.Queue()
        
    qu.put(root)
    count = 0
    maze2 = copy.deepcopy(maze)
    while not qu.empty():
      
      current = qu.get()
      current_i = current.anato_i_pos
      current_j = current.anato_j_pos
      count = count+1
      for row in current.state:
       print(''.join(str(cell) for cell in row))

      print()
      if maze2[current_i][current_j] == 'S':
        print(" found Safe House")
        return current.path
      if maze2[current_i][current_j] == 'w':
        current.has_weapon = True
        current.state = copy.deepcopy(remove_weapon(current_i,current_j, current.state))
        print(" found weapon")
        for row in current.state:
         print(''.join(str(cell) for cell in row))
      if maze2[current_i][current_j] == 'z':  
         if if_weapon(current) == True: #kill zombie here if has_weapon in node is true
               current.state = copy.deepcopy(kill_zombie(current_i,current_j,current.state))
         else: #kill_zombie is false to kill anato
             print("Anato killed!")
             return False 
      visited.add(tuple(map(tuple, current.state)))       
      newstateslist= new_states(current.state)   

      for st in newstateslist:
         if tuple(map(tuple, st)) not in visited:
             temp_node = Node(st)
            # print("in loop:")
             ai,aj = find_anto(st)
             temp_node.anato_i_pos, temp_node.anato_j_pos = ai,aj
             temp_node.parent = current
             temp_node.has_weapon = current.has_weapon
             temp_node.path = current.path + [st]
             qu.put(temp_node)
    return None  


def ucs(maze):
    priorityQueue = PriorityQueue()
    root = Node(maze)
    a_i, a_j = find_anto(maze)
    visited = set()
    root.anato_i_pos, root.anato_j_pos = a_i, a_j
    priorityQueue.put((0, root))
    count = 0
    maze2 = copy.deepcopy(maze)

    while not priorityQueue.empty():
       data = priorityQueue.get()
       current = data[1]
       current_i = current.anato_i_pos
       current_j = current.anato_j_pos
       count = count+1
       for row in current.state:
        print(''.join(str(cell) for cell in row))
      
       print()
       if maze2[current_i][current_j] == 'S':
        print("path cost = ", current.cost)
        print("number of nodes visited = ", count,"\n")
        print(" found Safe House")
        return current.path
       if maze2[current_i][current_j] == 'w':
        current.has_weapon = True
        current.state = copy.deepcopy(remove_weapon(current_i,current_j, current.state))
        print(" found weapon")
        for row in current.state:
         print(''.join(str(cell) for cell in row))
       if maze2[current_i][current_j] == 'z':  
         if if_weapon(current) == True: #kill zombie here if has_weapon in node is true
               current.state = copy.deepcopy(kill_zombie(current_i,current_j,current.state))
         else: #kill_zombie is false to kill anato
             print("Anato killed!")
             return False 
       visited.add(tuple(map(tuple, current.state)))       
       newstateslist= new_states(current.state)
       for st in newstateslist:
         if tuple(map(tuple, st)) not in visited:
             
             temp_node = Node(st)
            # print("in loop:")
             ai,aj = find_anto(st)
             temp_node.anato_i_pos, temp_node.anato_j_pos = ai,aj
             temp_node.parent = current
             temp_node.has_weapon = current.has_weapon
             temp_node.path = current.path + [st]
             temp_node.cost = current.cost +1
             priority = current.cost +1


             priorityQueue.put((priority,temp_node))







def if_weapon(node):
    if node.has_weapon == True:
      return True
    else: 
      return False

def kill_zombie(i,j,maze):

    i_coordinate = i
    j_coordinate = j

    maze[i_coordinate][j_coordinate] = 'A'

    return maze




def find_anto(maze):
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'A':
                print("anato coordinates = ",i,j)
                return i, j
    return None

def find_zombie(maze):
    zombie = []
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'z':
                zombie.append((i,j))
    return zombie

def find_safehouse(maze):
    safehouse = []
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'S':
                safehouse.append((i,j))
    return safehouse

def find_weapons(maze):
    weapon = []
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'w':
                weapon.append((i,j))
    return weapon

# Example usage:
filename = input("Enter the filename: ")
maze = read_map(filename)
# ma = remove_weapon(3,7,maze)
print("ANto coordinate : ")
ucs(maze)


Enter the filename: input.txt


FileNotFoundError: [Errno 2] No such file or directory: 'input.txt'